# Provides a Summary of Garmin Daily Monitoring Data

In [1]:
from IPython.display import display, Markdown
import snakemd

import fitfile
import datetime
from garmindb import GarminConnectConfigManager
from garmindb.garmindb import GarminDb, Attributes, Weight, Stress, RestingHeartRate, IntensityHR, Sleep
from garmindb.garmindb import MonitoringDb, Monitoring, MonitoringHeartRate, MonitoringIntensity, MonitoringClimb

from garmindb.garmindb import GarminSummaryDb, DaysSummary, MonitoringDb, MonitoringHeartRate, Sleep, GarminDb
from garmindb.summarydb import DaysSummary, SummaryDb

from jupyter_funcs import format_number

gc_config = GarminConnectConfigManager()
db_params_dict = gc_config.get_db_params()
garmin_db = GarminDb(db_params_dict)

# start date
start_ts = datetime.datetime.combine(datetime.date(year=2022, month=5, day=1), datetime.datetime.min.time())
# end date (today)
end_ts = datetime.datetime.combine(datetime.date.today(), datetime.datetime.max.time())


# sumDB data
sum_db = SummaryDb(db_params_dict, False)
data = DaysSummary.get_for_period(sum_db, start_ts, end_ts, DaysSummary)

measurement_system = Attributes.measurements_type(garmin_db)
unit_strings = fitfile.units.unit_strings[measurement_system]
distance_units = unit_strings[fitfile.units.UnitTypes.distance_long]

doc = snakemd.new_doc()

doc.add_heading("Monitoring Statistics - Totals")
doc.add_paragraph("Analysis of all monitoring data in the Garmin database.")

def __get_col_stats(table, col, name, ignore_le_zero=False, time_col=False):
    records = table.row_count(garmin_db)
    if time_col:
        maximum = table.get_time_col_max(garmin_db, col)
    else:
        maximum = format_number(table.get_col_max(garmin_db, col))
    if time_col:
        minimum = table.get_time_col_min(garmin_db, col)
    else:
        minimum = format_number(table.get_col_min(garmin_db, col, None, None, ignore_le_zero))
    if time_col:
        average = table.get_time_col_avg(garmin_db, col)
    else:
        average = format_number(table.get_col_avg(garmin_db, col, None, None, ignore_le_zero))
    if time_col:
        latest = table.get_col_latest(garmin_db, col)
    else:
        latest = format_number(table.get_col_latest(garmin_db, col))
    return [name, records, maximum, minimum, average, latest]

def __get_bb_stats():
    bb_min = format_number(garmin_sum_db.get_col_min(garmin_db, 'bb_min'))
    bb_max = format_number(garmin_sum_db.get_col_max(garmin_db, 'bb_max'))
    return [bb_min, bb_max]

doc.add_table(
    ['Name', 'Records', 'Maximum', 'Minimum', 'Average', 'Latest'],
    [
        __get_col_stats(Weight, Weight.weight, 'Weight'),
        __get_col_stats(Stress, Stress.stress, 'Stress', True),
        __get_col_stats(RestingHeartRate, RestingHeartRate.resting_heart_rate, 'RHR', True),
        __get_col_stats(Sleep, Sleep.total_sleep, 'Sleep', True, True),
        __get_col_stats(Sleep, Sleep.rem_sleep, 'REM Sleep', True, True),
       
    ])


doc.add_heading("Summary of Monitoring Statistics - by Stelios")
doc.add_paragraph("Analysis of other fitness data in the Garmin database.")



    # Handle bb_min and bb_max from garminsummarydb
   # if isinstance(table, GarminSummaryDb):
      #  bb_min = format_number(table.get_col_min(garmin_db, 'bb_min'))
     #   bb_max = format_number(table.get_col_max(garmin_db, 'bb_max'))
      #  return [name, records, maximum, minimum, average, latest, bb_min, bb_max]
  #  else:
   #     return [name, records, maximum, minimum, average, latest]

doc.add_heading("BB Statistics")
doc.add_table(
    ['BB Min', 'BB Max'],[]
    #[__get_bb_stats()]
)

def __get_monitoring_days(year):
    days = Monitoring.get_days(garmin_mon_db, year)
    days_count = len(days)
    if days_count > 0:
        first_day = days[0]
        last_day = days[-1]
        span = last_day - first_day + 1
    else:
        span = 0
    doc.add_paragraph(f"{year} Days with data ({days_count} count vs {span} span): {days}")
    for index in range(days_count - 1):
        day = int(days[index])
        next_day = int(days[index + 1])
        if next_day != day + 1:
            day_str = str(fitfile.conversions.day_of_the_year_to_datetime(year, day))
            next_day_str = str(fitfile.conversions.day_of_the_year_to_datetime(year, next_day))
            doc.add_paragraph(f"Days gap between {day} ({day_str}) and {next_day} ({next_day_str})")
    return days_count

def __get_monitoring_months(year):
    months = Monitoring.get_month_names(garmin_mon_db, year)
    doc.add_paragraph(f"{year} Months with data ({len(months)}): {months}")

doc.add_heading("Monitoring Records Coverage", 3)
doc.add_paragraph("This shows periods that data has been downloaded for. "
                    "Not seeing data for days you know Garmin has data? "
                    "Change the starting day and the number of days in GarminConnectConfig.json and do a full download.")
years = Monitoring.get_years(garmin_mon_db)
doc.add_paragraph(f"Monitoring records: {Monitoring.row_count(garmin_mon_db)}")
doc.add_paragraph(f"Monitoring Years with data ({len(years)}): {years}")
total_days = 0
for year in years:
    doc.add_heading(str(year), 4)
    __get_monitoring_months(year)
    total_days += __get_monitoring_days(year)
doc.add_paragraph(f"Total days with monitoring data: {total_days}")

display(Markdown(str(doc)))

NameError: name 'garmin_mon_db' is not defined